# Imports

In [1]:
import random
import pandas as pd
import numpy as np
import tensorflow as tf

from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, concatenate, Input, BatchNormalization
from tensorflow import keras
from sklearn.model_selection import train_test_split

# Connect drive

In [2]:
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


# Dataset

In [3]:
path_images="/content/drive/My Drive/uc project/images.npz"
dict_image_data = np.load(path_images)
image_data = dict_image_data['arr_0']

In [4]:
path_cities="/content/drive/My Drive/uc project/cities.npz"
dict_city_data = np.load(path_cities)
city_data = dict_city_data['arr_0']

path_class="/content/drive/My Drive/uc project/classes.npz"
dict_class_data = np.load(path_class)
class_data = dict_class_data['arr_0']

In [5]:
# all 512 images resized to 128
# data_512 = image_data[:,0][indices]
data_512 = image_data[:,0]
# all 512 images cropped to 256 images and resized to 128
# data_256 = image_data[:,1][indices]
data_256 = image_data[:,1]
# all 512 images cropped to 128 images
# data_128 = image_data[:,2][indices]
data_128 = image_data[:,2]

In [6]:
city_data.shape, data_512.shape, data_512.shape

((28195,), (28195, 128, 128, 3), (28195, 128, 128, 3))

Tranforming labels to onehot encoding

In [7]:
from sklearn.preprocessing import LabelBinarizer
label_as_binary = LabelBinarizer()

city_labels = label_as_binary.fit_transform(city_data)
class_labels = label_as_binary.fit_transform(class_data)

Split train and test data


In [8]:
train_512, test_512, train_256, test_256, train_128, test_128, train_city, test_city, train_class, test_class = train_test_split(data_512, data_256, data_128, city_labels, class_labels, test_size=0.2, random_state=42)

In [9]:
from collections import Counter

Counter(np.argmax(train_city, axis=1))

Counter({0: 4031, 1: 4340, 3: 6313, 2: 3678, 4: 4194})

In [10]:
Counter(np.argmax(test_city, axis=1))

Counter({2: 980, 1: 1126, 0: 991, 3: 1527, 4: 1015})

In [11]:
Counter(np.argmax(train_class, axis=1))

Counter({3: 1446,
         9: 909,
         0: 6836,
         4: 1520,
         13: 7428,
         11: 1185,
         10: 650,
         6: 608,
         17: 369,
         14: 117,
         16: 362,
         2: 99,
         15: 373,
         8: 255,
         5: 205,
         1: 132,
         7: 37,
         12: 25})

In [12]:
Counter(np.argmax(test_class, axis=1))

Counter({13: 1870,
         11: 338,
         4: 391,
         10: 171,
         9: 225,
         2: 28,
         0: 1681,
         6: 139,
         3: 332,
         14: 30,
         5: 55,
         17: 100,
         16: 89,
         15: 91,
         8: 54,
         1: 35,
         7: 7,
         12: 3})

# Networks

In [13]:
def basic_model(input_name):
  # 1
  model_input = Input(shape=(128, 128, 3), name=input_name)
  conv2d1 = Conv2D(filters=32, kernel_size=(3,3), activation='relu')(model_input)
  norm1 = BatchNormalization()(conv2d1)
  # 2
  pool1 = MaxPooling2D(pool_size=(2,2))(norm1)
  norm2 = BatchNormalization()(pool1)
  # 1
  conv2d2 = Conv2D(filters=32, kernel_size=(3,3), activation='relu')(norm2)
  norm3 = BatchNormalization()(conv2d2)
  # 2
  pool2 = MaxPooling2D(pool_size=(2,2))(norm3)
  norm4 = BatchNormalization()(pool2)
  # 15
  flat = Flatten()(norm4)
  dense = Dense(2048, activation='relu')(flat)
  model_output = Dropout(0.2)(dense)
  return model_input, model_output

## e128

Get input and initial output layers

In [14]:
basic_input, basic_output = basic_model('main_input')

Attach 2 outputs

In [15]:
#City
merged_hidden_city = Dense(2048, activation='relu')(basic_output)
merged_drop_city = Dropout(0.2)(merged_hidden_city)
city_output = Dense(len(np.unique(city_data)), activation='softmax', name='city_output')(merged_drop_city)

#Class
merged_hidden_class = Dense(2048, activation='relu')(basic_output)
merged_drop_class = Dropout(0.2)(merged_hidden_class)
class_output = Dense(len(np.unique(class_data)), activation='softmax', name='class_output')(merged_drop_class)

In [16]:
e128_model = Model(inputs=basic_input, outputs=[city_output, class_output])
# e128_model.compile(optimizer='adam', loss=['categorical_crossentropy', 'categorical_crossentropy'], metrics=['accuracy'])
gamma = 0.5
e128_model.compile(optimizer='adam',
                  loss={'city_output': 'categorical_crossentropy', 
                        'class_output': 'categorical_crossentropy'},
                  loss_weights={'city_output': gamma, 
                                'class_output': 1 - gamma}, 
                  metrics=['accuracy'])

In [17]:
e128_history = e128_model.fit(train_128, {'city_output': train_city, 'class_output': train_class},
                              # batch_size=20,
                              epochs=15,
                              shuffle=True,
                              validation_data=(test_128, {'city_output': test_city, 'class_output': test_class})
                 )

Epoch 1/15
705/705 [==============================] - 27s 34ms/step - loss: 1.7905 - city_output_loss: 1.3181 - class_output_loss: 2.2630 - city_output_accuracy: 0.6617 - class_output_accuracy: 0.4095 - val_loss: 1.4423 - val_city_output_loss: 0.9610 - val_class_output_loss: 1.9236 - val_city_output_accuracy: 0.5795 - val_class_output_accuracy: 0.4176
Epoch 2/15
705/705 [==============================] - 23s 33ms/step - loss: 1.0863 - city_output_loss: 0.6144 - class_output_loss: 1.5581 - city_output_accuracy: 0.7793 - class_output_accuracy: 0.4699 - val_loss: 2.3566 - val_city_output_loss: 2.6016 - val_class_output_loss: 2.1115 - val_city_output_accuracy: 0.4412 - val_class_output_accuracy: 0.3295
Epoch 3/15
705/705 [==============================] - 23s 33ms/step - loss: 0.9336 - city_output_loss: 0.5059 - class_output_loss: 1.3614 - city_output_accuracy: 0.8267 - class_output_accuracy: 0.5329 - val_loss: 2.4115 - val_city_output_loss: 2.7089 - val_class_output_loss: 2.1142 - val_cit

## e256

Get input and initial output layers

In [18]:
basic_input, basic_output = basic_model('main_input')

Attach 2 outputs

In [19]:
#City
merged_hidden_city = Dense(2048, activation='relu')(basic_output)
merged_drop_city = Dropout(0.2)(merged_hidden_city)
city_output = Dense(len(np.unique(city_data)), activation='softmax', name='city_output')(merged_drop_city)

#Class
merged_hidden_class = Dense(2048, activation='relu')(basic_output)
merged_drop_class = Dropout(0.2)(merged_hidden_class)
class_output = Dense(len(np.unique(class_data)), activation='softmax', name='class_output')(merged_drop_class)

In [20]:
e256_model = Model(inputs=basic_input, outputs=[city_output, class_output])
# e256_model.compile(optimizer='adam', loss=['categorical_crossentropy', 'categorical_crossentropy'], metrics=['accuracy'])
gamma = 0.5
e256_model.compile(optimizer='adam',
                  loss={'city_output': 'categorical_crossentropy', 
                        'class_output': 'categorical_crossentropy'},
                  loss_weights={'city_output': gamma, 
                                'class_output': 1 - gamma}, 
                  metrics=['accuracy'])

In [21]:
e256_history = e256_model.fit(train_256, {'city_output': train_city, 'class_output': train_class},
                              # batch_size=20,
                              epochs=15,
                              shuffle=True,
                              validation_data=(test_256, {'city_output': test_city, 'class_output': test_class})
                 )

Epoch 1/15
705/705 [==============================] - 25s 34ms/step - loss: 1.6482 - city_output_loss: 1.1993 - class_output_loss: 2.0971 - city_output_accuracy: 0.7169 - class_output_accuracy: 0.4229 - val_loss: 1.4530 - val_city_output_loss: 1.1509 - val_class_output_loss: 1.7551 - val_city_output_accuracy: 0.6592 - val_class_output_accuracy: 0.4151
Epoch 2/15
705/705 [==============================] - 23s 33ms/step - loss: 0.9828 - city_output_loss: 0.5002 - class_output_loss: 1.4655 - city_output_accuracy: 0.8290 - class_output_accuracy: 0.4878 - val_loss: 1.3071 - val_city_output_loss: 0.7294 - val_class_output_loss: 1.8848 - val_city_output_accuracy: 0.7732 - val_class_output_accuracy: 0.4056
Epoch 3/15
705/705 [==============================] - 23s 33ms/step - loss: 0.8128 - city_output_loss: 0.4017 - class_output_loss: 1.2239 - city_output_accuracy: 0.8690 - class_output_accuracy: 0.5765 - val_loss: 3.2982 - val_city_output_loss: 4.4117 - val_class_output_loss: 2.1847 - val_cit

## e512

Get input and initial output layers

In [22]:
basic_input, basic_output = basic_model('main_input')

Attach 2 outputs

In [23]:
#City
merged_hidden_city = Dense(2048, activation='relu')(basic_output)
merged_drop_city = Dropout(0.2)(merged_hidden_city)
city_output = Dense(len(np.unique(city_data)), activation='softmax', name='city_output')(merged_drop_city)

#Class
merged_hidden_class = Dense(2048, activation='relu')(basic_output)
merged_drop_class = Dropout(0.2)(merged_hidden_class)
class_output = Dense(len(np.unique(class_data)), activation='softmax', name='class_output')(merged_drop_class)

In [24]:
e512_model = Model(inputs=basic_input, outputs=[city_output, class_output])
# e512_model.compile(optimizer='adam', loss=['categorical_crossentropy', 'categorical_crossentropy'], metrics=['accuracy'])
gamma = 0.5
e512_model.compile(optimizer='adam',
                  loss={'city_output': 'categorical_crossentropy', 
                        'class_output': 'categorical_crossentropy'},
                  loss_weights={'city_output': gamma, 
                                'class_output': 1 - gamma}, 
                  metrics=['accuracy'])

In [25]:
e512_history = e512_model.fit(train_512, {'city_output': train_city, 'class_output': train_class},
                              # batch_size=20,
                              epochs=15,
                              shuffle=True,
                              validation_data=(test_512, {'city_output': test_city, 'class_output': test_class})
                 )

Epoch 1/15
705/705 [==============================] - 25s 34ms/step - loss: 1.6014 - city_output_loss: 1.1319 - class_output_loss: 2.0709 - city_output_accuracy: 0.7584 - class_output_accuracy: 0.4396 - val_loss: 1.4551 - val_city_output_loss: 1.2468 - val_class_output_loss: 1.6635 - val_city_output_accuracy: 0.6937 - val_class_output_accuracy: 0.4361
Epoch 2/15
705/705 [==============================] - 24s 34ms/step - loss: 0.8768 - city_output_loss: 0.3706 - class_output_loss: 1.3829 - city_output_accuracy: 0.8766 - class_output_accuracy: 0.5137 - val_loss: 1.8324 - val_city_output_loss: 1.7034 - val_class_output_loss: 1.9613 - val_city_output_accuracy: 0.5575 - val_class_output_accuracy: 0.3435
Epoch 3/15
705/705 [==============================] - 23s 33ms/step - loss: 0.7428 - city_output_loss: 0.3216 - class_output_loss: 1.1639 - city_output_accuracy: 0.8991 - class_output_accuracy: 0.5912 - val_loss: 3.3172 - val_city_output_loss: 3.3281 - val_class_output_loss: 3.3062 - val_cit

## final

Three inputs
  1. e128
  1. e256
  1. e512

Two outputs
  1. city
  1. class

Create the three basic input models

In [26]:
e128_input, e128_output = basic_model('e128_input')
e256_input, e256_output = basic_model('e256_input')
e512_input, e512_output = basic_model('e512_input')

Concatenating outputs of three models

In [27]:
merged = concatenate([e128_output, e256_output, e512_output])

Defining the outputs

In [28]:
#City
merged_hidden_city = Dense(2048, activation='relu')(merged)
merged_drop_city = Dropout(0.2)(merged_hidden_city)
city_output = Dense(len(np.unique(city_data)), activation='softmax', name='city_output')(merged_drop_city)

#Class
merged_hidden_class = Dense(2048, activation='relu')(merged)
merged_drop_class = Dropout(0.2)(merged_hidden_class)
class_output = Dense(len(np.unique(class_data)), activation='softmax', name='class_output')(merged_drop_class)

In [29]:
final_model = Model(inputs=[e128_input, e256_input, e512_input], outputs=[city_output, class_output])
# final_model.compile(optimizer='adam', loss=['categorical_crossentropy', 'categorical_crossentropy'], metrics=['accuracy'])
gamma = 0.5
final_model.compile(optimizer='adam',
                  loss={'city_output': 'categorical_crossentropy', 
                        'class_output': 'categorical_crossentropy'},
                  loss_weights={'city_output': gamma, 
                                'class_output': 1 - gamma}, 
                  metrics=['accuracy'])

Training model

In [30]:
final_history = final_model.fit({'e128_input': train_128, 'e256_input': train_256, 'e512_input': train_512}, {'city_output': train_city, 'class_output': train_class},
                                # batch_size=20,
                                epochs=15,
                                shuffle=True,
                                validation_data=({'e128_input': test_128, 'e256_input': test_256, 'e512_input': test_512}, {'city_output': test_city, 'class_output': test_class})
                 )
                

Epoch 1/15
705/705 [==============================] - 69s 95ms/step - loss: 1.9914 - city_output_loss: 1.4912 - class_output_loss: 2.4916 - city_output_accuracy: 0.7353 - class_output_accuracy: 0.4077 - val_loss: 1.1910 - val_city_output_loss: 0.7310 - val_class_output_loss: 1.6510 - val_city_output_accuracy: 0.7615 - val_class_output_accuracy: 0.4430
Epoch 2/15
705/705 [==============================] - 66s 93ms/step - loss: 1.0193 - city_output_loss: 0.4564 - class_output_loss: 1.5821 - city_output_accuracy: 0.8443 - class_output_accuracy: 0.4582 - val_loss: 1.2921 - val_city_output_loss: 0.8267 - val_class_output_loss: 1.7575 - val_city_output_accuracy: 0.7166 - val_class_output_accuracy: 0.4300
Epoch 3/15
705/705 [==============================] - 66s 93ms/step - loss: 0.9180 - city_output_loss: 0.3649 - class_output_loss: 1.4711 - city_output_accuracy: 0.8811 - class_output_accuracy: 0.4854 - val_loss: 1.2973 - val_city_output_loss: 0.8341 - val_class_output_loss: 1.7605 - val_cit

## Evaluate


In [31]:
y_e128_city_pred, y_e128_class_pred = e128_model.predict(test_128)
y_e256_city_pred, y_e256_class_pred = e256_model.predict(test_256)
y_e512_city_pred, y_e512_class_pred = e512_model.predict(test_512)
y_final_city_pred, y_final_class_pred = final_model.predict({'e128_input': test_128, 'e256_input': test_256, 'e512_input': test_512})

177/177 [==============================] - 3s 13ms/step


In [32]:
y_e128_city_pred = np.argmax(y_e128_city_pred, axis=1)
y_e256_city_pred = np.argmax(y_e256_city_pred, axis=1)
y_e512_city_pred = np.argmax(y_e512_city_pred, axis=1)
y_final_city_pred = np.argmax(y_final_city_pred, axis=1)

y_e128_class_pred = np.argmax(y_e128_class_pred, axis=1)
y_e256_class_pred = np.argmax(y_e256_class_pred, axis=1)
y_e512_class_pred = np.argmax(y_e512_class_pred, axis=1)
y_final_class_pred = np.argmax(y_final_class_pred, axis=1)

In [33]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

city_y = np.argmax(test_city, axis=1)
class_y = np.argmax(test_class, axis=1)

e128_city_cm = confusion_matrix(y_e128_city_pred, city_y)
e256_city_cm = confusion_matrix(y_e256_city_pred, city_y)
e512_city_cm = confusion_matrix(y_e512_city_pred, city_y)
final_city_cm = confusion_matrix(y_final_city_pred, city_y)

e128_class_cm = confusion_matrix(y_e128_class_pred, class_y)
e256_class_cm = confusion_matrix(y_e256_class_pred, class_y)
e512_class_cm = confusion_matrix(y_e512_class_pred, class_y)
final_class_cm = confusion_matrix(y_final_class_pred, class_y)

e128_city_f1 = f1_score(y_e128_city_pred, city_y, average='micro')
e256_city_f1 = f1_score(y_e256_city_pred, city_y, average='micro')
e512_city_f1 = f1_score(y_e512_city_pred, city_y, average='micro')
final_city_f1 = f1_score(y_final_city_pred, city_y, average='micro')

e128_class_f1 = f1_score(y_e128_class_pred, class_y, average='micro')
e256_class_f1 = f1_score(y_e256_class_pred, class_y, average='micro')
e512_class_f1 = f1_score(y_e512_class_pred, class_y, average='micro')
final_class_f1 = f1_score(y_final_class_pred, class_y, average='micro')

In [34]:
e128_city_cm

array([[ 754,   28,   61,   99,  289],
       [  88, 1026,  170,   82,  108],
       [  12,    5,  597,   21,   15],
       [ 133,   34,  127, 1265,  289],
       [   4,   33,   25,   60,  314]])

In [35]:
e256_city_cm

array([[ 382,    4,    9,   23,   29],
       [ 157, 1008,  226,  140,  208],
       [   9,   24,  582,  122,   63],
       [  66,    9,   42,  931,   46],
       [ 377,   81,  121,  311,  669]])

In [36]:
e512_city_cm

array([[ 534,   31,   20,    9,   36],
       [  54,  771,   27,   15,  190],
       [  62,  235,  803,  201,   96],
       [ 292,   67,  104, 1247,  244],
       [  49,   22,   26,   55,  449]])

In [37]:
final_city_cm

array([[ 759,   12,    7,   19,   33],
       [   7,  976,   38,   20,   22],
       [   9,   23,  777,   17,    9],
       [ 151,    6,   80, 1338,  126],
       [  65,  109,   78,  133,  825]])

In [38]:
e128_class_cm

array([[899,  12,   6,  27,  39,  10,  19,   1,  12,  49,  51,  42,   1,
        489,   2,  12,  16,  11],
       [  8,   4,   0,   0,   1,   0,   0,   0,   0,   0,   1,   2,   0,
          8,   1,   1,   1,   2],
       [ 34,   2,   2,   3,   6,   1,   0,   0,   0,   0,   3,  16,   0,
         36,   0,   5,   2,   1],
       [ 57,   5,   3, 131, 102,   6,  26,   0,   1,  11,  24,  53,   0,
        229,   5,   4,   8,   3],
       [ 33,   1,   2,  35,  52,   5,  17,   1,   0,  13,  13,  26,   0,
         87,   1,   3,  10,   2],
       [  8,   0,   3,   3,   9,   2,  10,   0,   0,   0,   3,   3,   0,
         22,   0,   0,   0,   0],
       [  8,   0,   0,   7,   9,   1,   2,   0,   1,   1,   2,   8,   0,
         24,   0,   2,   2,   0],
       [  2,   0,   0,   0,   1,   0,   0,   0,   0,   0,   0,   0,   0,
          4,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0],
       [  5,   1,   0,   0,   1,   0,

In [39]:
e256_class_cm

array([[1002,   14,   12,   47,   83,   16,   38,    0,   22,   56,   71,
          93,    1,  668,   15,   26,   36,   33],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],
       [  19,    1,    1,    3,    7,    3,    5,    0,    1,    0,    3,
          11,    0,   22,    0,    1,    1,    1],
       [   1,    0,    0,   40,   17,    3,    4,    0,    0,    0,    0,
           5,    0,   62,    0,    0,    0,    1],
       [  14,    0,    1,   46,   29,    4,    8,    0,    0,    0,    4,
          12,    0,   47,    0,    1,    2,    2],
       [   0,    0,    0,    0,    2,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],
       [  48,    1,    1,   13,   22,    2,    7,    0,    3,   18,    4,
          15,    0,   72,    1,    2,    4,    3],
       [   0,    0,    0,    0,    0,    0,    1,    0,    0,    0,    0,
           0,    0,    1,    0,    0,    0,    0],


In [40]:
e512_class_cm

array([[ 925,   13,   10,   16,   22,    6,   10,    0,   12,   39,   26,
          41,    0,  370,    7,    8,    8,    5],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],
       [   1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           2,    0,    3,    0,    0,    0,    0],
       [   3,    0,    0,   63,   17,    0,    0,    0,    0,    2,    3,
          16,    0,   68,    2,    1,    1,    0],
       [  18,    1,    2,   59,   75,    8,   27,    1,    1,    2,   15,
          55,    0,   97,    2,    1,   10,    3],
       [   4,    0,    0,    4,    4,    0,    1,    0,    0,    0,    0,
           1,    0,    7,    0,    0,    0,    0],
       [  16,    0,    0,    6,    7,    4,    3,    1,    1,    1,    2,
           3,    0,   36,    2,    2,    3,    0],
       [   0,    0,    0,    2,    1,    0,    0,    0,    0,    1,    0,
           1,    0,    4,    0,    1,    0,    0],


In [41]:
final_class_cm

array([[1166,   11,   12,    2,   11,    7,   11,    0,   27,   74,   45,
          36,    0,  606,    9,   21,   13,   18],
       [   0,    1,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    1,    0,
           0,    0,    3,    0,    0,    0,    0],
       [  19,    1,    2,  152,   57,    4,   10,    0,    1,    2,    4,
          44,    0,  176,    3,    2,    6,    1],
       [  53,    2,    3,  104,  216,   27,   81,    3,    2,    2,   34,
         143,    0,  228,    1,   13,   36,    3],
       [   0,    0,    0,    0,    1,    0,    0,    0,    0,    0,    0,
           0,    0,    1,    0,    0,    0,    0],
       [   3,    0,    1,    1,    8,    0,    1,    0,    0,    0,    2,
           2,    0,   14,    0,    0,    1,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],


In [42]:
print('e128 f1: city: {}, class: {}'.format(e128_city_f1, e128_class_f1))
print('e256 f1: city: {}, class: {}'.format(e256_city_f1, e256_class_f1))
print('e512 f1: city: {}, class: {}'.format(e512_city_f1, e512_class_f1))
print('final f1: city: {}, class: {}'.format(final_city_f1, final_class_f1))

e128 f1: city: 0.7015428267423302, class: 0.3637169710941656
e256 f1: city: 0.6334456463912042, class: 0.3294910445114382
e512 f1: city: 0.6745876928533427, class: 0.4021989714488384
final f1: city: 0.8290477034935272, class: 0.40946976414257846
